In [1]:
from os.path import isdir, join
import pickle
import numpy as np
import matplotlib.pyplot as plt
# import rosbag
import glob
from sklearn.model_selection import train_test_split, KFold
import os
import pdb
import math
import torch

In [2]:
pkg_path = '/home/zhe/HumanIntention'
dataset_filename = '2TargetsLabeled.p'
dataset_filepath = join(pkg_path, 'Dataset', dataset_filename)
with open(dataset_filepath,'rb') as f:
    data = pickle.load(f)

In [3]:
xtrain, xtest, ytrain, ytest = [], [], [], []
for intention in [0, 1]:
    intention_label = [intention for i in range(len(data[intention]))]
    xtraini, xtesti, ytraini, ytesti = train_test_split(data[intention], intention_label, test_size=5, random_state=0)
    xtrain += xtraini
    ytrain += ytraini
    xtest += xtesti
    ytest += ytesti

In [4]:
def process_data(raw_x, raw_y, obs_seq_len, pred_seq_len, skip=2):
    seq_len = obs_seq_len+pred_seq_len
    xobs, xpred, yintention = [], [], []
    for x, y in zip(raw_x, raw_y):
        n_seq = math.floor((len(x)-seq_len)/skip)+1
        idx_range = range(0, n_seq*skip, skip)
        for idx in idx_range:
            seq_data = x[idx:idx+seq_len]# (seq_len, 2)
            obs_seq_data, pred_seq_data = seq_data[:obs_seq_len], seq_data[obs_seq_len:]
            xobs.append(obs_seq_data)
            xpred.append(pred_seq_data)
            yintention.append(y)
    xobs = torch.stack(xobs, dim=0) # (N, obs_seq_len, 2)
    xpred = torch.stack(xpred, dim=0) # (N, pred_seq_len, 2)
    yintention = torch.tensor(yintention).unsqueeze(1).type(torch.float) # (N, 1)
    return xobs, xpred, yintention

skip = 2
obs_seq_len, pred_seq_len = 4, 6

xobs_train, xpred_train, yintention_train = process_data(xtrain, ytrain, obs_seq_len, pred_seq_len, skip=2)
xobs_test, xpred_test, yintention_test = process_data(xtest, ytest, obs_seq_len, pred_seq_len, skip=2)

In [6]:
torch.save({"xobs_train": xobs_train, "xpred_train": xpred_train, "yintention_train": yintention_train,\
            "xobs_test": xobs_test, "xpred_test": xpred_test, "yintention_test": yintention_test, \
            "obs_seq_len": obs_seq_len, "pred_seq_len": pred_seq_len, "skip": skip}, 'short_prediction_data.pt')

In [5]:
print(sum(yintention_train==0))
print(sum(yintention_train==1))

tensor([329])
tensor([469])


In [6]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for train_index, validation_index in kf.split(xobs_train):
#     print("TRAIN:", train_index, "VAL:", validation_index)
    print(xobs_train[train_index][20])
    print(sum(yintention_train[validation_index]==0))
    print(sum(yintention_train[validation_index]==1))
    print()
    break

tensor([[214., 215.],
        [221., 215.],
        [231., 215.],
        [235., 215.]])
tensor([61])
tensor([99])



In [15]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for train_index, validation_index in kf.split(xobs_train):
#     print("TRAIN:", train_index, "VAL:", validation_index)
    print(xobs_train[train_index][20])
    print(sum(yintention_train[validation_index]==0))
    print(sum(yintention_train[validation_index]==1))
    print()
    break

tensor([[214., 215.],
        [221., 215.],
        [231., 215.],
        [235., 215.]])
tensor([61])
tensor([99])



In [101]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
kf.get_n_splits()
# print(kf)


5

In [97]:
print(ytrain)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [98]:
for train_index, validation_index in kf.split(xtrain, ytrain):
    print("TRAIN:", train_index, "VAL:", validation_index)
    print(ytrain[train_index])

TRAIN: [ 0  1  3  5  6  7  8  9 12 13 14 15 16 17 18 19 20 21 23 24 25 26 29 30
 32 33 34 35 36 37 39 40 42 43 44 45 46 47 48 49] VAL: [ 2  4 10 11 22 27 28 31 38 41]


TypeError: only integer scalar arrays can be converted to a scalar index